In [12]:
import os, glob
import time
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from torchvision.models import resnet50

from torchsummary import summary
import torch_optimizer as optim

from sklearn.model_selection import train_test_split

In [13]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda:0


In [14]:
# 학습 데이터셋 폴더 경로를 지정해주세요.
train_dir = '/opt/ml/input/data/train'
trainimage_dir = os.path.join(train_dir, 'images')

In [15]:

# meta 데이터와 이미지 경로를 불러옵니다.
train_df = pd.read_csv(os.path.join(train_dir, 'train.csv'))

In [16]:

masks = ['mask1', 'mask2', 'mask3', 'mask4', 'mask5', 'incorrect_mask', 'normal']
wears = ['Wear', 'Wear', 'Wear', 'Wear', 'Wear', 'Incorrect', 'Not Wear']
mask_df = pd.DataFrame()
for person in train_df.values:
    for mask, wear in zip(masks, wears):
        mask_df = mask_df.append(pd.Series(np.append(person, (mask, wear))), ignore_index=True)
mask_df.columns = np.append(train_df.columns.values, ('mask', 'wear'))

In [17]:
print(mask_df)
mask_df = mask_df.sample(frac=1).reset_index(drop=True)

           id  gender   race   age                    path            mask  \
0      000001  female  Asian  45.0  000001_female_Asian_45           mask1   
1      000001  female  Asian  45.0  000001_female_Asian_45           mask2   
2      000001  female  Asian  45.0  000001_female_Asian_45           mask3   
3      000001  female  Asian  45.0  000001_female_Asian_45           mask4   
4      000001  female  Asian  45.0  000001_female_Asian_45           mask5   
...       ...     ...    ...   ...                     ...             ...   
18895  006959    male  Asian  19.0    006959_male_Asian_19           mask3   
18896  006959    male  Asian  19.0    006959_male_Asian_19           mask4   
18897  006959    male  Asian  19.0    006959_male_Asian_19           mask5   
18898  006959    male  Asian  19.0    006959_male_Asian_19  incorrect_mask   
18899  006959    male  Asian  19.0    006959_male_Asian_19          normal   

            wear  
0           Wear  
1           Wear  
2     

In [20]:
train, valid = train_test_split(mask_df, test_size=0.2, stratify=mask_df['wear'])

In [21]:
transform = transforms.Compose([
    transforms.CenterCrop(384),
    Resize((224, 224), Image.BILINEAR),
    
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

In [22]:
class GenderDataset(Dataset):
    def __init__(self, path, mask_df, transform):
        super(GenderDataset).__init__()
        self.path = path
        self.mask_df = mask_df
        self.transform = transform
        
    def __getitem__(self, idx):
        full_path = os.path.join(self.path, self.mask_df.iloc[idx]['path'])
        img_list = glob.glob(full_path + '/*')
        file_name = self.mask_df.iloc[idx]['mask']
        for img_name in img_list:
            if img_name.startswith(file_name):
                break
        image = Image.open(os.path.join(full_path, img_name))
        if self.transform:
            image = self.transform(image)
        
        label = self.mask_df.iloc[idx]['gender']
        label = 0 if label=='male' else 1
        return image, label
    
    def __len__(self):
        return len(self.mask_df)

In [23]:
gender_train_data = GenderDataset(trainimage_dir, train, transform)
gender_valid_data = GenderDataset(trainimage_dir, valid, transform)

In [24]:
batch_size = 128

In [25]:
gender_train = DataLoader(gender_train_data, batch_size=batch_size, shuffle=True, num_workers=2)
gender_valid = DataLoader(gender_valid_data, batch_size=batch_size, shuffle=True, num_workers=2)

In [26]:
class AgeDataset(Dataset):
    def __init__(self, path, mask_df, transform):
        super(AgeDataset).__init__()
        self.path = path
        self.mask_df = mask_df
        self.transform = transform
        
    def __getitem__(self, idx):
        full_path = os.path.join(self.path, self.mask_df.iloc[idx]['path'])
        img_list = glob.glob(full_path + '/*')
        file_name = self.mask_df.iloc[idx]['mask']
        for img_name in img_list:
            if img_name.startswith(file_name):
                break
        image = Image.open(os.path.join(full_path, img_name))
        if self.transform:
            image = self.transform(image)
        
        label = self.mask_df.iloc[idx]['age']
        if label >= 60.0:
            label = 2
        elif label >= 30.0:
            label = 1
        else:
            label = 0
        return image, label
    
    def __len__(self):
        return len(self.mask_df)

In [27]:
age_train_data = AgeDataset(trainimage_dir, train, transform)
age_valid_data = AgeDataset(trainimage_dir, valid, transform)

In [28]:
age_train = DataLoader(age_train_data, batch_size=batch_size, shuffle=True, num_workers=2)
age_valid = DataLoader(age_valid_data, batch_size=batch_size, shuffle=True, num_workers=2)

In [36]:
class MaskDataset(Dataset):
    def __init__(self, path, mask_df, transform):
        super(MaskDataset).__init__()
        self.path = path
        self.mask_df = mask_df
        self.transform = transform
        
    def __getitem__(self, idx):
        full_path = os.path.join(self.path, self.mask_df.iloc[idx]['path'])
        img_list = glob.glob(full_path + '/*')
        file_name = self.mask_df.iloc[idx]['mask']
        for img_name in img_list:
            if img_name.startswith(file_name):
                break
        image = Image.open(os.path.join(full_path, img_name))
        if self.transform:
            image = self.transform(image)
        
        label = self.mask_df.iloc[idx]['mask']
        if 'mask' in label:
            label = 0
            
        elif 'incorrect' in label:
            label = 1
            
        else:
            label = 2
            
        return image, label
    
    def __len__(self):
        return len(self.mask_df)

In [37]:
mask_train_data = MaskDataset(trainimage_dir, train, transform)
mask_valid_data = MaskDataset(trainimage_dir, valid, transform)


In [38]:
mask_train = DataLoader(mask_train_data, batch_size=batch_size, shuffle=True)
mask_valid = DataLoader(mask_valid_data, batch_size=batch_size, shuffle=True)

In [39]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        super(TestDataset).__init__()
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [40]:
test_dir = '/opt/ml/input/data/eval'

In [41]:
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
testimage_dir = os.path.join(test_dir, 'images')

In [42]:

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(testimage_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((224, 224), Image.BILINEAR),
    ToTensor(),
    
])
test_dataset = TestDataset(image_paths, transform)

test_loader = DataLoader(
    test_dataset,
    shuffle=False
)

In [43]:

model = resnet50(pretrained=True, progress=False)

In [44]:
class MyModel(nn.Module):
    def __init__(self, num_classes: int = 1000):
        super(MyModel, self).__init__()
        self.model = resnet50(pretrained=True, progress=False)
        
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)
        

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.model(x)
        
        return x

In [45]:
mask_model = MyModel(num_classes=3)
for param in mask_model.parameters():
    param.requires_grad = False # frozen
for param in mask_model.model.fc.parameters():
    param.requires_grad = True # 마지막 레이어 살리기
mask_model.to(device)

MyModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0

In [46]:
gender_model = MyModel(num_classes=2)
for param in gender_model.parameters():
    param.requires_grad = False # frozen
for param in gender_model.model.fc.parameters():
    param.requires_grad = True # 마지막 레이어 살리기
gender_model.to(device)

MyModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0

In [47]:

age_model = MyModel(num_classes=3)
for param in age_model.parameters():
    param.requires_grad = False # frozen
for param in age_model.model.fc.parameters():
    param.requires_grad = True # 마지막 레이어 살리기
age_model.to(device)

MyModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0

In [48]:
learning_rate = 0.001
betas = (0.9, 0.999)
weight_decay = 1e-4
T_max = 50

In [50]:
num_epochs = 30
optimizer = torch.optim.Adam(mask_model.parameters(), lr = 0.01)
lr_sched  = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer, 
milestones=[int(num_epochs * 0.5), int(num_epochs * 0.75)], gamma=0.1, last_epoch=-1)



valid_early_stop = 0
valid_best_loss = float('inf')
EARLY_STOPPING_EPOCH = 5
since = time.time()

final_train_loss = []
final_train_acc = []
final_valid_loss = []
final_valid_acc = []

for e in range(num_epochs) :
    print(f' ====================== epoch %d ======================' % (e+1) )
    train_loss_list = []
    train_acc_list = []
    

    # train
    mask_model.train()
    for i, (images, targets) in enumerate(mask_train) : 
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        scores = mask_model(images)
        _, preds = scores.max(dim=1)

        loss = F.cross_entropy(scores, targets)
        loss.backward()
        optimizer.step()
        lr_sched.step()
        correct = sum(targets == preds).cpu()
        acc=(correct/128 * 100)

        train_loss_list.append(loss)
        train_acc_list.append(acc)

        if i % 50 == 0 :
            print(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (i, loss, acc))

    train_mean_loss = np.mean(train_loss_list, dtype="float64")
    train_mean_acc = np.mean(train_acc_list, dtype="float64")

    final_train_loss.append(train_mean_loss)
    final_train_acc.append(train_mean_acc)

    epoch_time = time.time() - since
    since = time.time()

    print('')
    print(f'[Summary] Elapsed time : %.0f m %.0f s' % (epoch_time // 60, epoch_time % 60))
    print(f'Train Loss Mean %.4f | Accuracy %2.2f ' % (train_mean_loss, train_mean_acc) )

    # validation 
    mask_model.eval()
    valid_loss_list = []
    valid_acc_list = []
    for i, (images, targets) in enumerate(mask_valid) : 
        optimizer.zero_grad()
        images = images.to(device=device)
        targets = targets.to(device=device)

        with torch.no_grad():
            
            scores = mask_model(images)
            loss = F.cross_entropy(scores, targets)
            _, preds = scores.max(dim=1)

        correct = sum(targets == preds).cpu()
        acc=(correct/128 * 100)

        valid_loss_list.append(loss)
        valid_acc_list.append(acc)

    val_mean_loss = np.mean(valid_loss_list, dtype="float64")
    val_mean_acc = np.mean(valid_acc_list, dtype="float64")

    final_valid_loss.append(val_mean_loss)
    final_valid_acc.append(val_mean_acc)

    print(f'Valid Loss Mean %.4f | Accuracy %2.2f ' % (val_mean_loss, val_mean_acc) )
    print('')

    if val_mean_loss < valid_best_loss:
        valid_best_loss = val_mean_loss
        valid_early_stop = 0
        # new best model save (valid 기준)
        mask_best_model = mask_model
        path = './mask_model/'
        torch.save(mask_best_model.state_dict(), f'{path}model{val_mean_acc:2.2f}_epoch_{e}.pth')
    else:
        # early stopping    
        valid_early_stop += 1
        if valid_early_stop >= EARLY_STOPPING_EPOCH:
            print("EARLY STOPPING!!")
            break

    

 ====================== epoch 1 ======================
Iteration   0 | Train Loss  0.7714 | Classifier Accuracy 91.41
Iteration  50 | Train Loss  0.4281 | Classifier Accuracy 85.94
Iteration 100 | Train Loss  0.3486 | Classifier Accuracy 89.06

[Summary] Elapsed time : 1 m 45 s
Train Loss Mean 0.6656 | Accuracy 83.90 
Valid Loss Mean 0.4240 | Accuracy 84.38 

 ====================== epoch 2 ======================
Iteration   0 | Train Loss  0.4753 | Classifier Accuracy 82.81
Iteration  50 | Train Loss  0.3383 | Classifier Accuracy 89.84
Iteration 100 | Train Loss  0.4717 | Classifier Accuracy 82.81

[Summary] Elapsed time : 2 m 11 s
Train Loss Mean 0.4185 | Accuracy 85.08 
Valid Loss Mean 0.4223 | Accuracy 84.38 

 ====================== epoch 3 ======================
Iteration   0 | Train Loss  0.3603 | Classifier Accuracy 89.06
Iteration  50 | Train Loss  0.4457 | Classifier Accuracy 83.59
Iteration 100 | Train Loss  0.4515 | Classifier Accuracy 83.59

[Summary] Elapsed time : 2 m 12

In [53]:


optimizer = optim.RAdam(gender_model.parameters(), lr=learning_rate, betas=betas, weight_decay=weight_decay)
lr_sched = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=T_max)

num_epochs = 30

valid_early_stop = 0
valid_best_loss = float('inf')
EARLY_STOPPING_EPOCH = 5
since = time.time()

final_train_loss = []
final_train_acc = []
final_valid_loss = []
final_valid_acc = []

for e in range(num_epochs) :
    print(f' ====================== epoch %d ======================' % (e+1) )
    train_loss_list = []
    train_acc_list = []

    # train
    gender_model.train()
    for i, (images, targets) in enumerate(gender_train) : 
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        scores = gender_model(images)
        _, preds = scores.max(dim=1)

        loss = F.cross_entropy(scores, targets)
        loss.backward()
        optimizer.step()

        correct = sum(targets == preds).cpu()
        acc=(correct/128 * 100)

        train_loss_list.append(loss)
        train_acc_list.append(acc)

        if i % 50 == 0 :
            print(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (i, loss, acc))

    train_mean_loss = np.mean(train_loss_list, dtype="float64")
    train_mean_acc = np.mean(train_acc_list, dtype="float64")

    final_train_loss.append(train_mean_loss)
    final_train_acc.append(train_mean_acc)

    epoch_time = time.time() - since
    since = time.time()

    print('')
    print(f'[Summary] Elapsed time : %.0f m %.0f s' % (epoch_time // 60, epoch_time % 60))
    print(f'Train Loss Mean %.4f | Accuracy %2.2f ' % (train_mean_loss, train_mean_acc) )

    # validation 
    gender_model.eval()
    valid_loss_list = []
    valid_acc_list = []
    for i, (images, targets) in enumerate(gender_valid) : 
        optimizer.zero_grad()
        images = images.to(device=device)
        targets = targets.to(device=device)

        with torch.no_grad():
            scores = gender_model(images)
            loss = F.cross_entropy(scores, targets)
            _, preds = scores.max(dim=1)

        correct = sum(targets == preds).cpu()
        acc=(correct/128 * 100)

        valid_loss_list.append(loss)
        valid_acc_list.append(acc)

    val_mean_loss = np.mean(valid_loss_list, dtype="float64")
    val_mean_acc = np.mean(valid_acc_list, dtype="float64")

    final_valid_loss.append(val_mean_loss)
    final_valid_acc.append(val_mean_acc)

    print(f'Valid Loss Mean %.4f | Accuracy %2.2f ' % (val_mean_loss, val_mean_acc) )
    print('')

    if val_mean_loss < valid_best_loss:
        valid_best_loss = val_mean_loss
        valid_early_stop = 0
        # new best model save (valid 기준)
        gender_best_model = gender_model
        path = '/opt/ml/teamrepo/kbs/code/gender_model/'
        torch.save(gender_best_model.state_dict(), f'{path}model{val_mean_acc:2.2f}_epoch_{e}.pth')
    else:
        # early stopping    
        valid_early_stop += 1
        if valid_early_stop >= EARLY_STOPPING_EPOCH:
            print("EARLY STOPPING!!")
            break

    lr_sched.step()

 ====================== epoch 1 ======================
Iteration   0 | Train Loss  0.3074 | Classifier Accuracy 87.50
Iteration  50 | Train Loss  0.2670 | Classifier Accuracy 87.50
Iteration 100 | Train Loss  0.2751 | Classifier Accuracy 91.41

[Summary] Elapsed time : 0 m 48 s
Train Loss Mean 0.2673 | Accuracy 90.25 
Valid Loss Mean 0.2566 | Accuracy 89.48 

 ====================== epoch 2 ======================
Iteration   0 | Train Loss  0.2719 | Classifier Accuracy 88.28
Iteration  50 | Train Loss  0.2092 | Classifier Accuracy 93.75
Iteration 100 | Train Loss  0.1909 | Classifier Accuracy 92.97

[Summary] Elapsed time : 1 m 0 s
Train Loss Mean 0.2286 | Accuracy 91.16 
Valid Loss Mean 0.2252 | Accuracy 90.49 

 ====================== epoch 3 ======================
Iteration   0 | Train Loss  0.1224 | Classifier Accuracy 98.44
Iteration  50 | Train Loss  0.1630 | Classifier Accuracy 93.75
Iteration 100 | Train Loss  0.2000 | Classifier Accuracy 93.75

[Summary] Elapsed time : 1 m 1 s

In [54]:
optimizer = optim.RAdam(age_model.parameters(), lr=learning_rate, betas=betas, weight_decay=weight_decay)
lr_sched = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=T_max)

num_epochs = 30

valid_early_stop = 0
valid_best_loss = float('inf')
EARLY_STOPPING_EPOCH = 5
since = time.time()

final_train_loss = []
final_train_acc = []
final_valid_loss = []
final_valid_acc = []

for e in range(num_epochs) :
    print(f' ====================== epoch %d ======================' % (e+1) )
    train_loss_list = []
    train_acc_list = []

    # train
    age_model.train()
    for i, (images, targets) in enumerate(age_train) : 
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        scores = age_model(images)
        _, preds = scores.max(dim=1)

        loss = F.cross_entropy(scores, targets)
        loss.backward()
        optimizer.step()

        correct = sum(targets == preds).cpu()
        acc=(correct/128 * 100)

        train_loss_list.append(loss)
        train_acc_list.append(acc)

        if i % 50 == 0 :
            print(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (i, loss, acc))

    train_mean_loss = np.mean(train_loss_list, dtype="float64")
    train_mean_acc = np.mean(train_acc_list, dtype="float64")

    final_train_loss.append(train_mean_loss)
    final_train_acc.append(train_mean_acc)

    epoch_time = time.time() - since
    since = time.time()

    print('')
    print(f'[Summary] Elapsed time : %.0f m %.0f s' % (epoch_time // 60, epoch_time % 60))
    print(f'Train Loss Mean %.4f | Accuracy %2.2f ' % (train_mean_loss, train_mean_acc) )

    # validation 
    age_model.eval()
    valid_loss_list = []
    valid_acc_list = []
    for i, (images, targets) in enumerate(age_valid) : 
        optimizer.zero_grad()
        images = images.to(device=device)
        targets = targets.to(device=device)

        with torch.no_grad():
            scores = age_model(images)
            loss = F.cross_entropy(scores, targets)
            _, preds = scores.max(dim=1)

        correct = sum(targets == preds).cpu()
        acc=(correct/128 * 100)

        valid_loss_list.append(loss)
        valid_acc_list.append(acc)

    val_mean_loss = np.mean(valid_loss_list, dtype="float64")
    val_mean_acc = np.mean(valid_acc_list, dtype="float64")

    final_valid_loss.append(val_mean_loss)
    final_valid_acc.append(val_mean_acc)

    print(f'Valid Loss Mean %.4f | Accuracy %2.2f ' % (val_mean_loss, val_mean_acc) )
    print('')

    if val_mean_loss < valid_best_loss:
        valid_best_loss = val_mean_loss
        valid_early_stop = 0
        # new best model save (valid 기준)
        age_best_model = age_model
        path = '/opt/ml/teamrepo/kbs/code/age_model/'
        torch.save(age_best_model.state_dict(), f'{path}model{val_mean_acc:2.2f}_epoch_{e}.pth')
    else:
        # early stopping    
        valid_early_stop += 1
        if valid_early_stop >= EARLY_STOPPING_EPOCH:
            print("EARLY STOPPING!!")
            break

    lr_sched.step()

 ====================== epoch 1 ======================
Iteration   0 | Train Loss  1.1023 | Classifier Accuracy 38.28
Iteration  50 | Train Loss  0.7335 | Classifier Accuracy 76.56
Iteration 100 | Train Loss  0.6076 | Classifier Accuracy 80.47

[Summary] Elapsed time : 0 m 48 s
Train Loss Mean 0.7269 | Accuracy 71.50 
Valid Loss Mean 0.5236 | Accuracy 82.99 

 ====================== epoch 2 ======================
Iteration   0 | Train Loss  0.6262 | Classifier Accuracy 78.12
Iteration  50 | Train Loss  0.4957 | Classifier Accuracy 81.25
Iteration 100 | Train Loss  0.4558 | Classifier Accuracy 82.81

[Summary] Elapsed time : 1 m 0 s
Train Loss Mean 0.4544 | Accuracy 83.84 
Valid Loss Mean 0.4015 | Accuracy 83.98 

 ====================== epoch 3 ======================
Iteration   0 | Train Loss  0.3653 | Classifier Accuracy 88.28
Iteration  50 | Train Loss  0.3275 | Classifier Accuracy 85.16
Iteration 100 | Train Loss  0.3213 | Classifier Accuracy 84.38

[Summary] Elapsed time : 0 m 60 

In [55]:
mask_best_model.eval()
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
mask_predictions = []
for images in test_loader:
    with torch.no_grad():
        images = images.to(device)
        scores = mask_best_model(images)
        preds = scores.argmax(dim=-1)
        mask_predictions.extend(preds.cpu().numpy())

In [56]:
from collections import Counter
Counter(mask_predictions) #이야 더 확고해졌네
#레이블링 관련 알고리즘을 살펴보자. startswith의 대용은?


Counter({0: 12600})

In [57]:

gender_best_model.eval()
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
gender_predictions = []
for images in test_loader:
    with torch.no_grad():
        images = images.to(device)
        scores = gender_best_model(images)
        preds = scores.argmax(dim=-1)
        gender_predictions.extend(preds.cpu().numpy())

In [58]:
from collections import Counter
Counter(gender_predictions)

Counter({0: 4933, 1: 7667})

In [59]:
age_best_model.eval()
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
age_predictions = []
for images in test_loader:
    with torch.no_grad():
        images = images.to(device)
        scores = age_best_model(images)
        preds = scores.argmax(dim=-1)
        age_predictions.extend(preds.cpu().numpy())

In [60]:
from collections import Counter
Counter(age_predictions)

Counter({0: 2827, 1: 8740, 2: 1033})

In [61]:
all_predictions = []
size = len(submission)
class_map = np.array([[[0, 1, 2],
                       [3, 4, 5]],
                      [[6, 7, 8],
                       [9, 10, 11]],
                      [[12, 13, 14],
                       [15, 16, 17]]])
for idx in range(size):
    i = mask_predictions[idx]
    j = gender_predictions[idx]
    k = age_predictions[idx]
    all_predictions.append(class_map[i][j][k])

In [62]:
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission_baseline_pretrained.csv'), index=False)
print('test inference is done!')

test inference is done!


In [63]:
import datetime
now = datetime.datetime.now()
print(now)

2021-08-24 18:25:39.126484


In [64]:
print(now)

2021-08-24 18:25:39.126484
